In [1]:
import matplotlib.pyplot as plt
import numpy as np
import csv 
import pandas as pd
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier


percentualTreinamento = 0.7


dataset = {}
# Define the numeric labels to filter
labels={'M','B'}
labels_list = list(labels)


# Define column names based on dataset documentation
columns = [
    "ID", "Diagnosis",
    "Radius_mean", "Texture_mean", "Perimeter_mean", "Area_mean", "Smoothness_mean",
    "Compactness_mean", "Concavity_mean", "Concave_points_mean", "Symmetry_mean", "Fractal_dimension_mean",
    "Radius_se", "Texture_se", "Perimeter_se", "Area_se", "Smoothness_se",
    "Compactness_se", "Concavity_se", "Concave_points_se", "Symmetry_se", "Fractal_dimension_se",
    "Radius_worst", "Texture_worst", "Perimeter_worst", "Area_worst", "Smoothness_worst",
    "Compactness_worst", "Concavity_worst", "Concave_points_worst", "Symmetry_worst", "Fractal_dimension_worst"
]

# Load the dataset
dadosBrutos = pd.read_csv("../dataset/wdbc.data", names=columns, header=None)

# Drop the ID column (not useful for analysis)
dadosBrutos.drop(columns=["ID"], inplace=True)


# Filter rows where a column equals one of the labels
dadosBrutosLabel0 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[0]] 
dadosBrutosLabel1 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[1]] 

# metadata 
#print(dadosBrutos.head())



In [2]:
# Total number of samples

nAmostras_treinamento0 = len(dadosBrutosLabel0)
tamanhoTreinamento0 = int(nAmostras_treinamento0 * percentualTreinamento)

# Randomly select indices for group 1 - treinamento
indices_label0_treinamento = np.random.choice(dadosBrutosLabel0.index, size=tamanhoTreinamento0, replace=False)
# Select remaining indices for group 1 - teste
indices_label0_teste = dadosBrutosLabel0.index.difference(indices_label0_treinamento)

# Determine the size of group 2
nAmostras_treinamento1 = len(dadosBrutosLabel1)
tamanhoTreinamento1 = int(nAmostras_treinamento1 * percentualTreinamento)

# Randomly select indices for group 2 - treinamento
indices_label1_treinamento = np.random.choice(dadosBrutosLabel1.index, size=tamanhoTreinamento1, replace=False)
indices_label1_teste = dadosBrutosLabel0.index.difference(indices_label1_treinamento)

trainData = dadosBrutosLabel0.loc[indices_label0_treinamento]
buffer_trainData = dadosBrutosLabel1.loc[indices_label1_treinamento]
testData = dadosBrutosLabel0.loc[indices_label0_teste]
buffer_testData = dadosBrutosLabel1.loc[indices_label1_treinamento]

filtered_rows_Train = pd.concat([trainData,buffer_trainData], axis=0)
filtered_rows_Test = pd.concat([testData,buffer_testData], axis=0)

trainData = filtered_rows_Train.iloc[:, 1:31].to_numpy()
trainLabels= filtered_rows_Train.iloc[:, 0]
testData = filtered_rows_Test.iloc[:, 1:31].to_numpy()
testLabels= filtered_rows_Test.iloc[:, 0]

# aleatoriza ordem de treinamento
#indicesShuffledTranData = np.random.choice(trainData.index, size=1, replace=False)
#trainData = trainData.loc[indicesShuffledTranData]
#trainLabels = trainLabels.loc[indicesShuffledTranData]


trainLabels = trainLabels.replace({labels_list[0]: 0, labels_list[1]: 1}).to_numpy()
testLabels = testLabels.replace({labels_list[0]: 0, labels_list[1]: 1}) .to_numpy()

print(trainLabels)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


/var/folders/nf/hkgtx45s3f7_s6zv1l__8jw40000gp/T/ipykernel_62097/392804951.py:38: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  trainLabels = trainLabels.replace({labels_list[0]: 0, labels_list[1]: 1}).to_numpy()
/var/folders/nf/hkgtx45s3f7_s6zv1l__8jw40000gp/T/ipykernel_62097/392804951.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  testLabels = testLabels.replace({labels_list[0]: 0, labels_list[1]: 1}) .to_numpy()


In [3]:
percentualErrosMaximo=0.275
stepErros=0.01

vector = np.arange(0, percentualErrosMaximo, stepErros)
print("Blocos de erro entrada: ", vector, "Tamanho: ", len(vector))

Blocos de erro entrada:  [0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13
 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27] Tamanho:  28


## Experimento 1: Ruído distribuído de forma igual aos dois rótulos

In [4]:
import statistics
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


percentualErrosMaximo=0.275
stepErros=0.01

vector = np.arange(0, percentualErrosMaximo, stepErros)
print("Blocos de erro entrada: ", vector, "Tamanho: ", len(vector))

SEED = 42
np.random.seed(SEED)

classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators = 15,random_state = SEED), # n_estimators =100
    "SVM (RBF)": SVC(C=100,kernel='rbf', gamma=0.001,class_weight='balanced',random_state = SEED),
    "XGBoost": XGBClassifier(n_estimators = 20, objective = 'binary:logistic', tree_method = 'hist', eta = 0.1, # n_estimators = 100
                             max_depth = 3, enable_categorical = True,random_state = SEED),
    "MLP": MLPClassifier(hidden_layer_sizes=(50), activation='tanh', solver='sgd',
                    max_iter=500, alpha=0.001,random_state = SEED)
}
# MLPClassifier(hidden_layer_sizes=(50), activation='tanh', solver='lbfgs',
                    #max_iter=3000, alpha=0.001)

nRepeticoes=18
resultadosConsolidados_treino = []
resultadosConsolidados_teste = []
blocos = []
all_results=[]
resultadosAcc=[]


for nivelRuidoAtual in tqdm(vector):
    for indiceRepeticao in range(nRepeticoes):
        # Total number of samples

        dadosBrutosLabel0 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[0]] 
        dadosBrutosLabel1 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[1]] 

        nAmostras_treinamento0 = len(dadosBrutosLabel0)
        tamanhoTreinamento0 = int(nAmostras_treinamento0 * percentualTreinamento)
        tamanhoErroTreinamento0 = int(nAmostras_treinamento0 * (nivelRuidoAtual/2) )

        # Randomly select indices for group 1 - treinamento
        indices_label0_treinamento = np.random.choice(dadosBrutosLabel0.index, size=tamanhoTreinamento0, replace=False)
        indices_label0_erro =np.random.choice(indices_label0_treinamento, size=tamanhoErroTreinamento0, replace=False)

        # Select remaining indices for group 1 - teste
        indices_label0_teste = dadosBrutosLabel0.index.difference(indices_label0_treinamento)

        # Determine the size of group 2
        nAmostras_treinamento1 = len(dadosBrutosLabel1)
        tamanhoTreinamento1 = int(nAmostras_treinamento1 * percentualTreinamento)
        tamanhoErroTreinamento1 = int(nAmostras_treinamento0 * (nivelRuidoAtual/2) )

        # Randomly select indices for group 2 - treinamento
        indices_label1_treinamento = np.random.choice(dadosBrutosLabel1.index, size=tamanhoTreinamento1, replace=False)
        indices_label1_teste = dadosBrutosLabel0.index.difference(indices_label1_treinamento)
        indices_label1_erro =np.random.choice(indices_label1_treinamento, size=tamanhoErroTreinamento1, replace=False)

        # introduz erro
        dadosLabel0=dadosBrutosLabel0
        dadosLabel1=dadosBrutosLabel1
        dadosLabel0.loc[indices_label0_erro, "Diagnosis"] = dadosLabel0.loc[indices_label0_erro, "Diagnosis"].map({"M": "B", "B": "M"})
        dadosLabel1.loc[indices_label1_erro, "Diagnosis"] = dadosLabel1.loc[indices_label1_erro, "Diagnosis"].map({"M": "B", "B": "M"})

        trainData = dadosLabel0.loc[indices_label0_treinamento]
        buffer_trainData = dadosLabel1.loc[indices_label1_treinamento]
        testData = dadosLabel0.loc[indices_label0_teste]
        buffer_testData = dadosLabel1.loc[indices_label1_treinamento]

        filtered_rows_Train = pd.concat([trainData,buffer_trainData], axis=0)
        filtered_rows_Test = pd.concat([testData,buffer_testData], axis=0)

        #aleatoriza ordem de treinamento
        indicesShuffledTranData = np.random.choice(filtered_rows_Train.index, size=1, replace=False)
        trainData = filtered_rows_Train.loc[indicesShuffledTranData]
        trainLabels = filtered_rows_Train.loc[indicesShuffledTranData]


        trainData = filtered_rows_Train.iloc[:, 1:31]
        trainLabels= filtered_rows_Train.iloc[:, 0]
        testData = filtered_rows_Test.iloc[:, 1:31]
        testLabels= filtered_rows_Test.iloc[:, 0]


        trainLabels = filtered_rows_Train.loc[:, "Diagnosis"].map({"M": 0, "B": 1}).to_numpy()
        testLabels = filtered_rows_Test.loc[:, "Diagnosis"].map({"M": 0, "B": 1}).to_numpy()
          
        # Calcula o nivel de ruido real baseado no número de indices flipados divididio pelo numero total de indices 
        nivelRuidoAtual_real=(len(indices_label0_erro)+len(indices_label1_erro))/len(trainLabels)
        
        ### Treinamentos
        for name, model in classifiers.items():
            model.fit(trainData, trainLabels)
            resultadoTreinamento=model.predict(trainData)
            resultadoTeste=model.predict(testData)
            acc_teste = ( sum((testLabels==resultadoTeste))/len(testLabels) )
            resultadosAcc.append([nivelRuidoAtual_real, name, indiceRepeticao,acc_teste])
            for true_label, pred_label in zip(resultadoTeste, testLabels):
                all_results.append([nivelRuidoAtual_real, name, indiceRepeticao, true_label, pred_label, acc_teste])

        
      
    # print("observados_treinamento=",observados_treinamento)
    # print("soma=",sum(trainLabels==observados_treinamento))æ
    # print("len=",len(trainLabels))

    # print(sum(trainLabels==observados_treinamento)/len(trainLabels))

print(blocos)

df_results = pd.DataFrame(all_results, columns=["Noise Level", "Classifier", "IndexRep", "True Label", "Predicted Label", "Accuracy"])
resultadosExp = pd.DataFrame(resultadosAcc, columns=["Noise Level", "Classifier", "IndexRep", "Accuracy"])


Blocos de erro entrada:  [0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13
 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27] Tamanho:  28


100%|██████████| 28/28 [03:13<00:00,  6.91s/it]


[]


In [6]:
import os
print(resultadosExp)
string = "../resultados/experimento/resultadosExperimento1.csv" 
if not os.path.exists(string):
    resultadosExp.to_csv(string, index=True)


      Noise Level     Classifier  IndexRep  Accuracy
0        0.000000  Random Forest         0  0.984026
1        0.000000      SVM (RBF)         0  0.977636
2        0.000000        XGBoost         0  0.984026
3        0.000000            MLP         0  0.904153
4        0.000000  Random Forest         1  0.980831
...           ...            ...       ...       ...
2011     0.141058            MLP        16  0.859425
2012     0.141058  Random Forest        17  0.964856
2013     0.141058      SVM (RBF)        17  0.948882
2014     0.141058        XGBoost        17  0.884984
2015     0.141058            MLP        17  0.814696

[2016 rows x 4 columns]


## Experimento 2: Médico tendencioso a classificar como tumor maligno

In [7]:
import statistics
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


percentualErrosMaximo=0.275
stepErros=0.01

vector = np.arange(0, percentualErrosMaximo, stepErros)
print("Blocos de erro entrada: ", vector, "Tamanho: ", len(vector))

SEED = 42
np.random.seed(SEED)

classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators = 15,random_state = SEED), # n_estimators =100
    "SVM (RBF)": SVC(C=100,kernel='rbf', gamma=0.001,class_weight='balanced',random_state = SEED),
    "XGBoost": XGBClassifier(n_estimators = 20, objective = 'binary:logistic', tree_method = 'hist', eta = 0.1, # n_estimators = 100
                             max_depth = 3, enable_categorical = True,random_state = SEED),
    "MLP": MLPClassifier(hidden_layer_sizes=(50), activation='tanh', solver='sgd',
                    max_iter=500, alpha=0.001,random_state = SEED)
}
# MLPClassifier(hidden_layer_sizes=(50), activation='tanh', solver='lbfgs',
                    #max_iter=3000, alpha=0.001)

nRepeticoes=18
resultadosConsolidados_treino = []
resultadosConsolidados_teste = []
blocos = []
all_results=[]
resultadosAcc=[]


for nivelRuidoAtual in tqdm(vector):
    for indiceRepeticao in range(nRepeticoes):
        # Total number of samples

        dadosBrutosLabel0 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[0]] # maligno
        dadosBrutosLabel1 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[1]] 

        nAmostras_treinamento0 = len(dadosBrutosLabel0)
        tamanhoTreinamento0 = int(nAmostras_treinamento0 * percentualTreinamento)
        tamanhoErroTreinamento0 = 0 # tendencioso a classificar maligno

        # Randomly select indices for group 1 - treinamento
        indices_label0_treinamento = np.random.choice(dadosBrutosLabel0.index, size=tamanhoTreinamento0, replace=False)
        indices_label0_erro =np.random.choice(indices_label0_treinamento, size=tamanhoErroTreinamento0, replace=False)

        # Select remaining indices for group 1 - teste
        indices_label0_teste = dadosBrutosLabel0.index.difference(indices_label0_treinamento)

        # Determine the size of group 2
        nAmostras_treinamento1 = len(dadosBrutosLabel1)
        tamanhoTreinamento1 = int(nAmostras_treinamento1 * percentualTreinamento)
        tamanhoErroTreinamento1 = int(nAmostras_treinamento0 * (nivelRuidoAtual) ) # tendencioso a classificar maligno

        # Randomly select indices for group 2 - treinamento
        indices_label1_treinamento = np.random.choice(dadosBrutosLabel1.index, size=tamanhoTreinamento1, replace=False)
        indices_label1_teste = dadosBrutosLabel0.index.difference(indices_label1_treinamento)
        indices_label1_erro =np.random.choice(indices_label1_treinamento, size=tamanhoErroTreinamento1, replace=False)

        # introduz erro
        dadosLabel0=dadosBrutosLabel0
        dadosLabel1=dadosBrutosLabel1
        dadosLabel0.loc[indices_label0_erro, "Diagnosis"] = dadosLabel0.loc[indices_label0_erro, "Diagnosis"].map({"M": "B", "B": "M"})
        dadosLabel1.loc[indices_label1_erro, "Diagnosis"] = dadosLabel1.loc[indices_label1_erro, "Diagnosis"].map({"M": "B", "B": "M"})

        trainData = dadosLabel0.loc[indices_label0_treinamento]
        buffer_trainData = dadosLabel1.loc[indices_label1_treinamento]
        testData = dadosLabel0.loc[indices_label0_teste]
        buffer_testData = dadosLabel1.loc[indices_label1_treinamento]

        filtered_rows_Train = pd.concat([trainData,buffer_trainData], axis=0)
        filtered_rows_Test = pd.concat([testData,buffer_testData], axis=0)

        #aleatoriza ordem de treinamento
        indicesShuffledTranData = np.random.choice(filtered_rows_Train.index, size=1, replace=False)
        trainData = filtered_rows_Train.loc[indicesShuffledTranData]
        trainLabels = filtered_rows_Train.loc[indicesShuffledTranData]


        trainData = filtered_rows_Train.iloc[:, 1:31]
        trainLabels= filtered_rows_Train.iloc[:, 0]
        testData = filtered_rows_Test.iloc[:, 1:31]
        testLabels= filtered_rows_Test.iloc[:, 0]


        trainLabels = filtered_rows_Train.loc[:, "Diagnosis"].map({"M": 0, "B": 1}).to_numpy()
        testLabels = filtered_rows_Test.loc[:, "Diagnosis"].map({"M": 0, "B": 1}).to_numpy()


        # Calcula o nivel de ruido real baseado no número de indices flipados divididio pelo numero total de indices 
        nivelRuidoAtual_real=(len(indices_label0_erro)+len(indices_label1_erro))/len(trainLabels)
        
        ### Treinamentos
        for name, model in classifiers.items():
            model.fit(trainData, trainLabels)
            resultadoTreinamento=model.predict(trainData)
            resultadoTeste=model.predict(testData)
            acc_teste = ( sum((testLabels==resultadoTeste))/len(testLabels) )
            resultadosAcc.append([nivelRuidoAtual_real, name, indiceRepeticao,acc_teste])
            for true_label, pred_label in zip(resultadoTeste, testLabels):
                all_results.append([nivelRuidoAtual_real, name, indiceRepeticao, true_label, pred_label, acc_teste])

        
      
    # print("observados_treinamento=",observados_treinamento)
    # print("soma=",sum(trainLabels==observados_treinamento))æ
    # print("len=",len(trainLabels))

    # print(sum(trainLabels==observados_treinamento)/len(trainLabels))

print(blocos)

df_results2 = pd.DataFrame(all_results, columns=["Noise Level", "Classifier", "IndexRep", "True Label", "Predicted Label", "Accuracy"])
resultadosExp2 = pd.DataFrame(resultadosAcc, columns=["Noise Level", "Classifier", "IndexRep", "Accuracy"])


Blocos de erro entrada:  [0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13
 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27] Tamanho:  28


100%|██████████| 28/28 [03:14<00:00,  6.95s/it]


[]


In [8]:
import os
print(resultadosExp2)
string = "../resultados/experimento/resultadosExperimento2.csv" 
if not os.path.exists(string):
    resultadosExp2.to_csv(string, index=True)

      Noise Level     Classifier  IndexRep  Accuracy
0        0.000000  Random Forest         0  0.984026
1        0.000000      SVM (RBF)         0  0.977636
2        0.000000        XGBoost         0  0.984026
3        0.000000            MLP         0  0.904153
4        0.000000  Random Forest         1  0.980831
...           ...            ...       ...       ...
2011     0.143577            MLP        16  0.769968
2012     0.143577  Random Forest        17  0.964856
2013     0.143577      SVM (RBF)        17  0.971246
2014     0.143577        XGBoost        17  0.840256
2015     0.143577            MLP        17  0.731629

[2016 rows x 4 columns]


## Experimento 3: Médico tendencioso a classificar como tumor benigno

In [9]:
import statistics
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


percentualErrosMaximo=0.275
stepErros=0.01

vector = np.arange(0, percentualErrosMaximo, stepErros)
print("Blocos de erro entrada: ", vector, "Tamanho: ", len(vector))

SEED = 42
np.random.seed(SEED)

classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators = 15,random_state = SEED), # n_estimators =100
    "SVM (RBF)": SVC(C=100,kernel='rbf', gamma=0.001,class_weight='balanced',random_state = SEED),
    "XGBoost": XGBClassifier(n_estimators = 20, objective = 'binary:logistic', tree_method = 'hist', eta = 0.1, # n_estimators = 100
                             max_depth = 3, enable_categorical = True,random_state = SEED),
    "MLP": MLPClassifier(hidden_layer_sizes=(50), activation='tanh', solver='sgd',
                    max_iter=500, alpha=0.001,random_state = SEED)
}
# MLPClassifier(hidden_layer_sizes=(50), activation='tanh', solver='lbfgs',
                    #max_iter=3000, alpha=0.001)

nRepeticoes=18
resultadosConsolidados_treino = []
resultadosConsolidados_teste = []
blocos = []
all_results=[]
resultadosAcc=[]


for nivelRuidoAtual in tqdm(vector):
    for indiceRepeticao in range(nRepeticoes):
        # Total number of samples

        dadosBrutosLabel0 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[0]] # maligno
        dadosBrutosLabel1 = dadosBrutos[dadosBrutos['Diagnosis'] == labels_list[1]] # benigno

        nAmostras_treinamento0 = len(dadosBrutosLabel0)
        tamanhoTreinamento0 = int(nAmostras_treinamento0 * percentualTreinamento)
        tamanhoErroTreinamento0 = int(nAmostras_treinamento0 * (nivelRuidoAtual) )

        # Randomly select indices for group 1 - treinamento
        indices_label0_treinamento = np.random.choice(dadosBrutosLabel0.index, size=tamanhoTreinamento0, replace=False)
        indices_label0_erro =np.random.choice(indices_label0_treinamento, size=tamanhoErroTreinamento0, replace=False)

        # Select remaining indices for group 1 - teste
        indices_label0_teste = dadosBrutosLabel0.index.difference(indices_label0_treinamento)

        # Determine the size of group 2
        nAmostras_treinamento1 = len(dadosBrutosLabel1)
        tamanhoTreinamento1 = int(nAmostras_treinamento1 * percentualTreinamento)
        tamanhoErroTreinamento1 = 0

        # Randomly select indices for group 2 - treinamento
        indices_label1_treinamento = np.random.choice(dadosBrutosLabel1.index, size=tamanhoTreinamento1, replace=False)
        indices_label1_teste = dadosBrutosLabel0.index.difference(indices_label1_treinamento)
        indices_label1_erro =np.random.choice(indices_label1_treinamento, size=tamanhoErroTreinamento1, replace=False)

        # introduz erro
        dadosLabel0=dadosBrutosLabel0
        dadosLabel1=dadosBrutosLabel1
        dadosLabel0.loc[indices_label0_erro, "Diagnosis"] = dadosLabel0.loc[indices_label0_erro, "Diagnosis"].map({"M": "B", "B": "M"})
        dadosLabel1.loc[indices_label1_erro, "Diagnosis"] = dadosLabel1.loc[indices_label1_erro, "Diagnosis"].map({"M": "B", "B": "M"})

        trainData = dadosLabel0.loc[indices_label0_treinamento]
        buffer_trainData = dadosLabel1.loc[indices_label1_treinamento]
        testData = dadosLabel0.loc[indices_label0_teste]
        buffer_testData = dadosLabel1.loc[indices_label1_treinamento]

        filtered_rows_Train = pd.concat([trainData,buffer_trainData], axis=0)
        filtered_rows_Test = pd.concat([testData,buffer_testData], axis=0)

        #aleatoriza ordem de treinamento
        indicesShuffledTranData = np.random.choice(filtered_rows_Train.index, size=1, replace=False)
        trainData = filtered_rows_Train.loc[indicesShuffledTranData]
        trainLabels = filtered_rows_Train.loc[indicesShuffledTranData]
        

        trainData = filtered_rows_Train.iloc[:, 1:31]
        trainLabels= filtered_rows_Train.iloc[:, 0]
        testData = filtered_rows_Test.iloc[:, 1:31]
        testLabels= filtered_rows_Test.iloc[:, 0]


        trainLabels = filtered_rows_Train.loc[:, "Diagnosis"].map({"M": 0, "B": 1}).to_numpy()
        testLabels = filtered_rows_Test.loc[:, "Diagnosis"].map({"M": 0, "B": 1}).to_numpy()

        # Calcula o nivel de ruido real baseado no número de indices flipados divididio pelo numero total de indices 
        nivelRuidoAtual_real=(len(indices_label0_erro)+len(indices_label1_erro))/len(trainLabels)
        
        ### Treinamentos
        for name, model in classifiers.items():
            model.fit(trainData, trainLabels)
            resultadoTreinamento=model.predict(trainData)
            resultadoTeste=model.predict(testData)
            acc_teste = ( sum((testLabels==resultadoTeste))/len(testLabels) )
            resultadosAcc.append([nivelRuidoAtual_real, name, indiceRepeticao,acc_teste])
            for true_label, pred_label in zip(resultadoTeste, testLabels):
                all_results.append([nivelRuidoAtual_real, name, indiceRepeticao, true_label, pred_label, acc_teste])

        
      
    # print("observados_treinamento=",observados_treinamento)
    # print("soma=",sum(trainLabels==observados_treinamento))æ
    # print("len=",len(trainLabels))

    # print(sum(trainLabels==observados_treinamento)/len(trainLabels))

print(blocos)

df_results3 = pd.DataFrame(all_results, columns=["Noise Level", "Classifier", "IndexRep", "True Label", "Predicted Label", "Accuracy"])
resultadosExp3 = pd.DataFrame(resultadosAcc, columns=["Noise Level", "Classifier", "IndexRep", "Accuracy"])


Blocos de erro entrada:  [0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13
 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24 0.25 0.26 0.27] Tamanho:  28


100%|██████████| 28/28 [03:11<00:00,  6.84s/it]


[]


In [11]:
import os
print(resultadosExp3)
string = "../resultados/experimento/resultadosExperimento3.csv" 
if not os.path.exists(string):
    resultadosExp3.to_csv(string, index=True)

      Noise Level     Classifier  IndexRep  Accuracy
0        0.000000  Random Forest         0  0.984026
1        0.000000      SVM (RBF)         0  0.977636
2        0.000000        XGBoost         0  0.984026
3        0.000000            MLP         0  0.904153
4        0.000000  Random Forest         1  0.980831
...           ...            ...       ...       ...
2011     0.143577            MLP        16  0.795527
2012     0.143577  Random Forest        17  0.923323
2013     0.143577      SVM (RBF)        17  0.865815
2014     0.143577        XGBoost        17  0.907348
2015     0.143577            MLP        17  0.795527

[2016 rows x 4 columns]
